# Spark Cluster Slave Nodes termination

### Importing Required Libraries ::
* __boto3__: Required to connect to AWS
* __json__: To convert python native dictionaries to string, to write in files
* __botocore__: Required to handle the exceptions related to boto3 tasks

In [1]:
import boto3
import json
from botocore.exceptions import ClientError

### Reading User defines properties::
* User is allowed to provide specific configurations using provided format of configuration file. If user does not provide any confguration or provides wrong configuration format, then default values will be used. Please check **README.MD** file for default values.

In [2]:
try:
    with open("cluster_config.json", "r") as config_file:
        user_config = json.load(config_file)

    region = user_config.get('Region', "us-east-1")
    user = user_config.get('UserName', "root")
    project_tag = user_config.get('ProjectTag', "SparkCluster")
except Exception as e:
    print("Unexpected error while fetching available status: " + str(e))
    exit()

### Creating boto3 session, clients and resources ::

In [3]:
try:
    session = boto3.session.Session(region_name=region)
    ec2_client = session.client('ec2')
    ec2_resource = session.resource('ec2')
except ClientError as e:
    print("Unexpected error while creating boto3 session, client and resources: " + str(e))
    exit()

### Extract details of running Slave Nodes for current user ::
* Extracted slave nodes will be terminated one by one.
* If there are any running processes, those will be terminated automatically.

In [4]:
try:
    instances = ec2_resource.instances.filter(
        Filters=[
            {
                'Name': 'instance-state-name',
                'Values': ['running']
            },
            {
                'Name': 'tag:Project',
                'Values': [project_tag]
            },
            {
                'Name': 'tag:User',
                'Values': [user]
            },
            {
                'Name': 'tag:NodeType',
                'Values': ['Slave']
            }
        ]
    )
except ClientError as e:
    print("Unexpected error while looking for already running Master node EC2 instance for user-'" + user + "': " + str(e))
    exit()

### Terminating the EC2 for master node on AWS ::
* __terminate__ API is used under EC2 resource to terminate the extracted EC2 servers(Slave Nodes of out Spark Cluster).

In [5]:
if list(instances):
    try:
        for instance in instances:
            instance.terminate()
            print("Slave node Instance('" + str(instance.id) + "') for User('" + user + "') is pushed to termination. It will be terminated within 2-3 mins.")
            
    except ClientError as e:
        print("Unexpected error while trying to terminate the Slave node EC2 servers for User('" + user + "'): " + str(e))
        exit()
else:
    print("There are no running slave nodes for user-'" + user + "'.")

Slave node Instance('i-0f7ebf9ec6108e5c4') for User('ccbp-dev-user-saumalya') is pushed to termination. It will be terminated within 2-3 mins.
Slave node Instance('i-0a29289898713fb3d') for User('ccbp-dev-user-saumalya') is pushed to termination. It will be terminated within 2-3 mins.
Slave node Instance('i-073c8767a836eb1b0') for User('ccbp-dev-user-saumalya') is pushed to termination. It will be terminated within 2-3 mins.
